In [26]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.svm import SVC
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.neural_network import MLPClassifier
from gensim.models import Word2Vec

import pandas as pd
import numpy as np
import scipy.sparse as sp
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import scipy.sparse as sp
import re
from sklearn.model_selection import train_test_split


In [54]:
df_tweets = pd.read_csv("dframe.csv")
df_ids = pd.read_csv("CMU_MisCov19_dataset.csv")
df = pd.merge(df_tweets, df_ids, left_on='id', right_on='status_id')
df = df[['text','id','status_id','annotation1','annotation2']]
df.head()

,text,id,status_id,annotation1,annotation2
0,"News you can use, from a public health officia...",1236988271625936896,1236988271625936896,calling out or correction,NaN
1,#Twimbos are you aware of the fake COVID-19 he...,1236985619047419904,1236985619047419904,calling out or correction,calling out or correction
2,"No, lime juice won't immunize you: Some people...",1222615296344608768,1222615296344608768,calling out or correction,NaN
3,#Coronavirus Where to next? https://t.co/Xssp4...,1236981003224113152,1236981003224113152,news,ambiguous or hard to classify
4,Does cocaine help with coronavirus? WHO has th...,1236980398598557696,1236980398598557696,calling out or correction,NaN


In [59]:

wnl = WordNetLemmatizer()
def lemmaSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(wnl.lemmatize(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)



In [60]:
pattern = re.compile(r"[A-Z][a-z]+|\d+|[A-Z]+(?![a-z])")

def split_hashtag(tag):
    return pattern.findall(tag)

In [67]:
split_hashtag("@RealDonaldTrump")

['Real', 'Donald', 'Trump']

In [70]:

df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|,;]')
BAD_SYMBOLS_RE = re.compile('[^2-8a-z +_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text) 
    text = text.replace('x', '')
    ext = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE) #remove url
    text = lemmaSentence(text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwords from text
    return text

In [71]:
df['text'] = df['text'].apply(clean_text)
df['text']

0       news use public health official interviewed np...
1       twimbos aware fake covid health tip ignore acc...
2       lime juice wont immunize people wrongly seem t...
3       coronavirus net http tco ssp4es65k http tco ct...
4       doe cocaine help coronavirus ha answer speak o...
                              ...                        
4203    friend sent u interview er doctor seems found ...
4204    wrong trump continues push pressure governor r...
4205               could cure covid still get 2 like post
4206    cnet even coronavirus derail 5gs global moment...
4207    realdonaldtrump vp tuckercarlson seanhannity m...
Name: text, Length: 4208, dtype: object

In [72]:
def tfidf_vectorizer(reviews_train, reviews_test = None):
    tf_vectorizer = TfidfVectorizer(analyzer = str.split, tokenizer = str.split, preprocessor = lambda x : x)
    tf_train = tf_vectorizer.fit_transform(reviews_train)
    features = list(tf_vectorizer.vocabulary_.keys())
    features.sort()
    
    tf_test = None
    if reviews_test is not None:
        tf_test = tf_vectorizer.transform(reviews_test)
        
    return (tf_train, tf_test, features)

In [73]:
tf,test, features = tfidf_vectorizer(df['text'])
n_topics = len(np.unique(df['annotation1']))
n_top_words = 3
lda = LatentDirichletAllocation(n_components=n_topics, learning_method='online', random_state=0)
lda.fit(tf)
top_words = []
features = np.array(features)
for topic in range(n_topics):
    sorted_indices = np.argsort(lda.components_[topic])
    top_words.append(list(features[sorted_indices[-n_top_words:]]))
    
top_words

[['bullet', 'youd', 'code'],
 ['wuhan', 'china', 'bioweapon'],
 ['stake', 'peru', 'student'],
 ['owns', 'condom', 'november'],
 ['contracting', 'stopping', 'demand'],
 ['strengthens', '4times', 'cuz'],
 ['denial', 'catching', 'google'],
 ['vahappened', 'crowd', 'antiva'],
 ['http', 'tco', 'covid'],
 ['colloidal', 'silver', 'company'],
 ['warn', 'book', 'class'],
 ['http', 'tco', 'tower'],
 ['newsthats', 'elected', 'rising'],
 ['23', 'ministry', 'shortage'],
 ['sunshine', 'roll', 'coronavirushttps'],
 ['department', 'environment', 'politico']]

In [74]:
X = df['text']
X_train, X_test, features = tfidf_vectorizer(X[train_indices], X[test_indices])
print(features)

['+', '+234734532434', '+8635388783email', '+general', '+hang', '+more', '+take', '+youre', '+zinc', '2', '2+', '2+6+2+66', '22', '22224254', '22224388senatemajldr', '2222525', '2245622', '225', '22apocalypse', '22coronavirus', '22election', '22espaola', '22if', '22mg', '22protests', '22report', '22watch', '23', '23+', '24', '243ukpfmnj', '25', '25+', '25huge', '25its', '25qanon', '25th', '25thamendment', '26', '263chat', '26tvwsqa', '27', '275cwsnhw', '28', '28iynsmesu', '2ai44eunpy', '2b', '2bcfmkeiw', '2cnktpcu', '2covid', '2day', '2daylockdown', '2dglrdp7k', '2dyptepuoe', '2em5vmzjl', '2ffvjiktme', '2fphgliua', '2fv7daqazu', '2g', '2gvc2g8fge', '2gycrbiht', '2how', '2jzba_e_pakistan', '2k+', '2lnqqyoolh', '2m', '2mcdmmoic', '2merkm3mo', '2mg', '2min', '2ncov', '2nd', '2ndwave', '2nutekduka', '2nylcavlq', '2part', '2pblhoqyit', '2r4euvcce', '2rj6tnbeuf', '2rtkkpvk', '2somethings', '2ssya2nl6', '2sylkfpamk', '2ufcizuyz', '2uvmmkpro', '2vatammn7v', '2vctpjgro', '2voebotbo6', '2vtyuwvd

In [79]:
y = df['annotation1']
X = df['text']


skf = StratifiedKFold(n_splits=10, shuffle=True,random_state=0)


model = SVC(kernel = "rbf",C=10)
for train_indices, test_indices in skf.split(X, y):
    X_train, X_test, features = tfidf_vectorizer(X[train_indices], X[test_indices])

    y_train = y[train_indices]
    y_test = y[test_indices]


    model.fit(X_train, y_train)

C:\Python36\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


In [80]:
model.predict(X_test)

array(['calling out or correction', 'calling out or correction',
       'conspiracy', 'calling out or correction', 'politics',
       'conspiracy', 'calling out or correction',
       'calling out or correction', 'calling out or correction',
       'calling out or correction', 'calling out or correction',
       'calling out or correction', 'calling out or correction',
       'calling out or correction', 'true prevention',
       'calling out or correction', 'calling out or correction',
       'calling out or correction', 'true public health response',
       'calling out or correction', 'conspiracy',
       'calling out or correction', 'conspiracy',
       'false fact or prevention', 'sarcasm or satire',
       'calling out or correction', 'conspiracy',
       'calling out or correction', 'calling out or correction',
       'calling out or correction', 'calling out or correction',
       'calling out or correction', 'conspiracy',
       'calling out or correction', 'calling out or cor